In [1]:
# Necessary imports
from datetime import datetime
import pytz
from copy import deepcopy
from matplotlib import pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import json
import os

from acnportal import acnsim, algorithms
from acnportal.acnsim import analysis
from acnportal.signals.tariffs import TimeOfUseTariff
from adacharge import *

### Experiment Setupu parameters
Followings are some necessary paramters for the experiment (for simplicity extracted directly)

In [2]:
API_KEY = 'DEMO_TOKEN'
TIMEZONE = pytz.timezone('America/Los_Angeles')
SITE = 'caltech'
PERIOD = 5  # minutes
VOLTAGE = 208  # volts
KW_TO_AMPS = 1000 / 208
KWH_TO_AMP_PERIODS = KW_TO_AMPS * (60 / 5)
MAX_LEN = 144
FORCE_FEASIBLE = True
EVENTS_DIR = 'events/'
VERBOSE = True

In [3]:
def get_events(start, end, ideal_battery, force_feasible, max_len):
    """ Gather Events from ACN-Data with a local cache. """
    event_name = f'{start}:{end}:{ideal_battery}:{force_feasible}:' \
                 f'{max_len}'
    path = os.path.join(EVENTS_DIR, event_name + '.json')
    if os.path.exists(path):
        with open(path, 'r') as f:
            return acnsim.EventQueue.from_json(f)
    start_time = TIMEZONE.localize(datetime.strptime(start, '%m-%d-%Y'))
    end_time = TIMEZONE.localize(datetime.strptime(end, '%m-%d-%Y'))
    default_battery_power = 6.656
    if ideal_battery:
        battery_params=None
    else:
        battery_params={'type': acnsim.Linear2StageBattery,
                        'capacity_fn': acnsim.models.battery.batt_cap_fn}
    events = acnsim.acndata_events.generate_events(API_KEY, SITE,
                                                   start_time, end_time,
                                                   PERIOD, VOLTAGE,
                                                   default_battery_power,
                                                   force_feasible=force_feasible,
                                                   max_len=max_len,
                                                   battery_params=battery_params)
    if not os.path.exists(EVENTS_DIR):
        os.mkdir(EVENTS_DIR)
    with open(path, 'w') as f:
        events.to_json(f)
    return events

### Creating a Wrapper Experiment Class

In [4]:
class Experiment:
    """ Wrapper for ACN-Sim Experiments including caching serialized experiment to disk. """
    def __init__(self, sim):
        self.sim = sim

    def calc_metrics(self):
        """ Calculate metrics from simulation. """
        metrics = {
            'proportion_delivered': analysis.proportion_of_energy_delivered(
                self.sim) * 100,
            'demands_fully_met': analysis.proportion_of_demands_met(
                self.sim) * 100,
            'peak_current': self.sim.peak,
            'demand_charge': analysis.demand_charge(self.sim),
            'energy_cost': analysis.energy_cost(self.sim),
            'total_energy_delivered': analysis.total_energy_delivered(self.sim),
            'total_energy_requested': analysis.total_energy_requested(self.sim)
        }
        return metrics

    def log_local_file(self, path):
        """ Write simulation, metrics and solver statistics to disk. """
        self.sim.to_json(path + 'sim.json')
        with open(path + 'metrics.json', 'w') as outfile:
            json.dump(self.calc_metrics(), outfile)
        with open(path + 'solve_stats.json', 'w') as outfile:
            json.dump(self.sim.scheduler.solve_stats, outfile)

    def run_and_store(self, path):
        """ Run experiment and store results. """
        print(f'Starting - {path}')
        if os.path.exists(path + 'sim.json'):
            print(f'Already Run - {path}...')
            return
        try:
            self.sim.run()
            if not os.path.exists(path):
                os.makedirs(path)
            self.log_local_file(path)
            print(f'Done - {path}')
        except Exception as e:
            print(f'Failed - {path}')
            print(e)

### Simulation configuration 

In [5]:
def configure_sim(
    alg, 
    cap, 
    start, 
    events, 
    basic_evse=True,
    estimate_max_rate=False, 
    uninterrupted_charging=False,
    quantized=False,
    allow_overcharging=False,
    tariff_name=None,
    offline=False
):
    """ Configure simulation. """
    start_time = TIMEZONE.localize(datetime.strptime(start, '%m-%d-%Y'))
        
    if estimate_max_rate:
        alg.max_rate_estimator = algorithms.SimpleRampdown()
        alg.estimate_max_rate = True
    alg.uninterrupted_charging = uninterrupted_charging
    alg.allow_overcharging = allow_overcharging

    # Some algorithms support a quantized option
    if quantized:
        try:
            alg.quantize = True
        except:
            pass
        try:
            alg.reallocate = True
        except:
            pass

    cn = acnsim.sites.caltech_acn(voltage=VOLTAGE,
                                  transformer_cap=cap,
                                  basic_evse=basic_evse)
    if tariff_name is not None:
        signals = {'tariff': TimeOfUseTariff(tariff_name)}
    else:
        signals = {}
    sim = acnsim.Simulator(cn, alg, events, start_time, signals=signals,
                           period=PERIOD, verbose=False)
    
    if offline:
        alg.register_events(events)
        alg.solve()
    
    return sim

### Experiment Configuration
**Note**: For simplicity extracted directly from source

In [8]:
start = '9-1-2018'
end = '10-1-2018'
tariff_name = 'sce_tou_ev_4_march_2019'
revenue = 0.3
# Scenario I is the offline optimal.
scenarios = {
                'II': {'ideal_battery': True,
                      'estimate_max_rate': False,
                      'uninterrupted_charging': False,
                      'quantized': False,
                      'basic_evse': True
                },
                'III': {'ideal_battery': True,
                       'estimate_max_rate': False,
                       'uninterrupted_charging': True,
                       'quantized': True,
                       'basic_evse': False
                },
                # 'IV': {'ideal_battery': False,
                #         'estimate_max_rate': True,
                #         'uninterrupted_charging': False,
                #         'quantized': False,
                #         'basic_evse': True
                # },
                # 'V': {'ideal_battery': False,
                #        'estimate_max_rate': True,
                #        'uninterrupted_charging': True,
                #        'quantized': True,
                #        'basic_evse': False
                # },
}

### Energy Delivered with constrained infrastructure 

In [9]:
energy_del_base_dir = 'results/infrastructure_utilization_results'

### Running Simulation
- Maximizing total energy delivered when infrastructure is oversubscribed
- Electricity prices are static or user statisfaction is the primary concern

Utility Function:
$$
    {U^{QC}(r)} := u^{QC}(r) + 10^{-12}u^{ES}(r)
$$

Here $U^{QC}$ encourages the system to deliver energy as quickly as possible, which helps free capacity for future arrivals. The regularizer ${u^{ES}(r)}$ promotes equal sharing between similar EVs and force a unique solution.

In [12]:
ALGS = dict()
# ALGS['LLF'] = algorithms.SortedSchedulingAlgo(algorithms.least_laxity_first)
# ALGS['EDF'] = algorithms.SortedSchedulingAlgo(algorithms.earliest_deadline_first)
ALGS['RR'] = algorithms.RoundRobin(algorithms.first_come_first_served, continuous_inc=1)

quick_charge_obj = [
    ObjectiveComponent(quick_charge),
    ObjectiveComponent(equal_share, 1e-12)
]
ALGS['ASA-QC'] = AdaptiveSchedulingAlgorithm(quick_charge_obj,
                                             solver='MOSEK',
                                             max_recompute=1)

In [14]:
# Online Algorithms
caps = list(range(20, 81, 10)) + [150]
for scenario_id, scenario in scenarios.items():
    for cap in caps:
        for alg in ['ASA-QC', 'RR']:
            output_dir = f'{energy_del_base_dir}/{start}:{end}/{scenario_id}/{cap}/{alg}/'
            events = get_events(start, end, scenario['ideal_battery'], FORCE_FEASIBLE, MAX_LEN)

            sim = configure_sim(
                cap=cap,
                alg=deepcopy(ALGS[alg]),
                start=start,
                events=events,
                basic_evse=scenario['basic_evse'],
                estimate_max_rate=scenario['estimate_max_rate'],
                uninterrupted_charging=scenario['uninterrupted_charging'],
                quantized=scenario['quantized'],
                tariff_name=tariff_name
            )
            ex = Experiment(sim)
            ex.run_and_store(output_dir)

Starting - results/infrastructure_utilization_results/9-1-2018:10-1-2018/II/20/ASA-QC/


/home/codespace/.python/current/lib/python3.10/site-packages/cvxpy/problems/problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
